In [15]:
import re
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition, ensemble
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle


In [16]:
trainDF=pd.read_csv("clean.csv",encoding='mac_roman', na_filter=True, na_values='[]')
trainDF.dropna(inplace=True)

In [17]:
trainDF.head()

,Unnamed: 0,content,Labels,text_clean,text_tokens,text_tokens_lemma
0,0,Turns out the cop did not even bake that fucki...,1,turns cop even bake fucking cake,"['turns', 'cop', 'even', 'bake', 'fucking', 'c...","['turn', 'cop', 'even', 'bake', 'fucking', 'ca..."
1,1,@realDonaldTrump Hey Don! You up? Donnnnaaald!...,0,hey donnnnaaald awake,"['hey', 'donnnnaaald', 'awake']","['hey', 'donnnnaaald', 'awake']"
2,2,Man drowned in Nichols home is 5th hurricane d...,1,man drowned nichols home th hurricane death sc...,"['man', 'drowned', 'nichols', 'home', 'th', 'h...","['man', 'drowned', 'nichols', 'home', 'th', 'h..."
3,3,It‚Äö√Ñ√¥s time to phase immigration down so w...,1,time phase immigration phase americans back,"['time', 'phase', 'immigration', 'phase', 'ame...","['time', 'phase', 'immigration', 'phase', 'ame..."
4,4,Calm is a superpower. ‚Äö√∫√• https://t.co/B...,1,calm superpower,"['calm', 'superpower']","['calm', 'superpower']"


In [18]:
X=trainDF['text_tokens_lemma']
Y=trainDF['Labels']
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, Y,test_size=0.2)

In [19]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text_tokens_lemma'])
pickle.dump(count_vect, open('cv.pkl','wb'))
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [20]:
xtrain_count

<66128x44572 sparse matrix of type '<class 'numpy.int64'>'
	with 525597 stored elements in Compressed Sparse Row format>

In [21]:
loreg=linear_model.LogisticRegression(max_iter=200)# initialize the model
loreg.fit(xtrain_count, train_y) # fit he model
pickle.dump(loreg, open('model.pkl','wb'))
y_pred=loreg.predict(xvalid_count) # now predict
cm = confusion_matrix(valid_y, y_pred)
print(cm)
accuracy_score(valid_y, y_pred)    

[[  806   643]
 [  208 14876]]


0.9485271880481462

In [22]:
#lower case it
#take input
#remove everything apart from a-zA-Z
#remove numbers
#tokenize it
#remove stopwords
#lemmetize it
#pass lemmetized sentence through count vectors
#pass through model
#get prediction

In [ ]:
cv = pickle.load(open('cv.pkl','rb'))
model = pickle.load(open('model.pkl','rb'))
review=input("Enter a tweet:")
wordnet = WordNetLemmatizer()
corpus = []
review = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", '', review)
review = re.sub(r"\d+", '', review)
review = review.lower()
review= nltk.word_tokenize(review)
review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
review = ' '.join(review)
corpus.append(review)
print(corpus)
corpus = cv.transform(corpus)
print(model.predict(corpus))

In [24]:
if(model==0):
    print("Human Tweeted")
else:
    print("Bot Tweeted")

Bot Tweeted
